# Version information

In [2]:
from datetime import date
print("Running date:", date.today().strftime("%B %d, %Y"))
import pyleecan
print("Pyleecan version:" + pyleecan.__version__)
import SciDataTool
print("SciDataTool version:" + SciDataTool.__version__)

Running date: March 18, 2022
Pyleecan version:1.3.7
SciDataTool version:1.4.24


# MeshSolution 객체를 사용하는 방법
MeshSolution 모듈은 메쉬에 연결된 데이터를 저장, 작업 수행 및 표시할 수 있는 코드의 일부입니다. 예를 들어, FEMM의 정자기 솔루션은 이 모듈을 사용하여 저장됩니다.

이 튜토리얼은 MeshSolution 모듈의 현재 기능이 허용하는 다양한 가능성을 보여줍니다. 현재는 주로 FEMM으로 계산된 2D 전자기장을 저장하는 데 전념하지만 목표는 Pyleecan 내부에 포함될 수 있는 모든 물리학으로 일반화하는 것입니다.
 
이 튜토리얼과 관련된 노트북은 [GitHub](https://github.com/Eomys/pyleecan/tree/master/Tutorials/tuto_MeshSolution.ipynb)에서 볼 수 있습니다.

이 튜토리얼은 이 모듈이 어떻게 작동하는지 깊이 이해하고 잠재적으로 코드 개발에 기여하고자 하는 사람들을 위한 것입니다.
# How to use MeshSolution objects
The MeshSolution module is the part of the code which allows to store, perform operation(s), and display data linked to a mesh. For example, the magnetostatic solution from FEMM is stored using this module.

This tutorial shows the different possibilities allowed by the current features of the MeshSolution module. At the moment, it is mainly dedicated to store 2D electromagnetic fields computed with FEMM, but the goal is to generalize to any physics that could be included inside Pyleecan.
 
The notebook related to this tutorial is available on [GitHub](https://github.com/Eomys/pyleecan/tree/master/Tutorials/tuto_MeshSolution.ipynb).

This tutorial is for people who wish to understand in depth how this module works, and potentially contribute to the development of the code.


## Architecture

기본 클래스는 MeshSolution입니다. 두 가지 중요한 속성이 있습니다.
- Mesh 클래스의 인스턴스 목록입니다. Mesh 클래스를 사용하면 메쉬의 매개변수(좌표, 연결 ...)를 저장할 수 있습니다.
- Solution 클래스의 인스턴스 목록입니다. Solution 클래스는 메쉬와 관련된 솔루션 필드를 저장할 수 있습니다.

따라서 MeshSolution을 사용하면 다양한 종류의 메쉬와 솔루션을 연결할 수 있습니다.
The main class is the MeshSolution. It has two important attributes:
- A list of instances of Mesh class. The Mesh class allows to store the parameters of the mesh (coordinates, connectivity ...).
- A list of instances of Solution class. The Solution class allows to store solution field related to a mesh.

Thus, the MeshSolution allows to make the link between the different kind of meshes and solutions.


### MeshMat 개체 정의

모든 기능은 Pyleecan에서 자동으로 초기화/정의되어야 하지만 MeshSolution 모듈의 기본 원리를 소개하기 위해 일부 개체를 직접 정의할 것입니다.

### Defining a MeshMat object

Although every features should be automatically initialized/defined in Pyleecan, we are going to define by hand some of the objects in order to introduce the basics principle of the MeshSolution module.

## Defining a SolutionMat object and plot
The MeshSolution object allows to make the link between data (such as FE results) and the corresponding mesh stored in a Mesh object. Thus, all the plot and post-processing methods should be available in the MeshSolution class.

The main available post-processing are the plots (such as plot_contour and plot_glyph).

Here is an example with plot_contour: a scalar field is defined by giving its values all points of the mesh. 

In [9]:
import numpy as np
from pyleecan.Classes.SolutionMat import SolutionMat
from pyleecan.Classes.SolutionVector import SolutionVector
from pyleecan.Classes.SolutionData import SolutionData

from pyleecan.definitions import TEST_DIR
from pyleecan.Classes.ImportMeshMat import ImportMeshMat
import meshio
from os.path import join
from pyleecan.definitions import TEST_DIR
from pyleecan.Classes.ImportMeshMat import ImportMeshMat
import meshio
from os.path import join
from pyleecan.Classes.MeshMat import MeshMat
from pyleecan.Classes.NodeMat import NodeMat
from pyleecan.Classes.CellMat import CellMat
from pyleecan.Classes.MeshSolution import MeshSolution


축 개념을 사용하면 SciDataTool 개체에서와 같이 값을 올바르게 추출할 수 있습니다. -> SolutionMat/SolutionData/SolutionVector에서 메서드를 호출하는 것과 같은 방식입니다.

SolutionMat을 사용하면 추가 축 "구성요소"를 사용하여 벡터 필드를 정의할 수도 있습니다.

The notion of axis allows to correctly extract values as it would be with SciDataTool objects -> same way to call methods in SolutionMat/SolutionData/SolutionVector. 

Using SolutionMat, one can also defined a vector field by using an additional axis "component".

이 예에서 2D 필드는 3D 메쉬에 정의됩니다. 실제로 메쉬와 필드에는 고유한 "차원" 속성이 있습니다. 가능한 경우 메모리 공간을 제한할 수 있습니다.

In this example, a 2D field is defined on a 3D mesh. Indeed, the mesh and the field have distinct "dimension" attributes. It enables to limit the memory space when possible. 

## 외부 메쉬 가져오기

현재 Pyleecan은 주로 meshio 라이브러리에 의존하여 모든 유형의 메쉬 파일을 pyvista에서 읽을 수 있는 .vtk로 변환합니다. 이 주제에 대한 모든 기여를 환영합니다. 그러나 최근에 .unv 파일을 가져오는 방법을 추가했습니다.

## Import an external Mesh

At the moment, Pyleecan mainly relies on the meshio librairy to convert any type of mesh file into a .vtk which is readable by pyvista. Any contribution on this topic is welcome. However, we have recently added a method to import .unv files.

In [21]:
mesh_obj = ImportMeshMat(
    file_path=join(TEST_DIR, join('C:/Users/KDH2018-PC/Downloads', 'Jmag_export.unv')),
)
mesh = mesh_obj.get_data()
    
# Import in Pyleecan with MeshVTK
MS = MeshSolution(mesh=[mesh])
MS.plot_mesh()

None


In [20]:
Test_mesh=MS.get_mesh()
anumber_mesh=Test_mesh.get_node()


In [49]:
#print(mesh.get_node())
#print(mesh.get_cell())
print(mesh.get_node_indice)


<bound method get_node_indice of <pyleecan.Classes.MeshMat.MeshMat object at 0x000001AD3F9C75B0>>


array([1.000e+00, 2.000e+00, 3.000e+00, ..., 6.963e+03, 6.964e+03,
       6.965e+03])

## Import external Solution data


In [11]:
import scipy.io as scio
field = scio.loadmat('Z:/01_Codes_Projects/Pyleecan_fork/Tutorials/force_vector.mat')
force_array=field['force_array']

#vector = np.ones((1,len(force_array),2))
#
#force_array=np.transpose(force_array)



print(np.shape(force_array))
type(force_array)

(6126, 3)


numpy.ndarray

In [12]:
force_array=np.delete(force_array,2,axis=1)
print(np.shape(force_array))

print(len(force_array))
print(np.shape(force_array))
temp=force_array.reshape((1,len(force_array),2))
print(np.shape(temp))

(6126, 2)
6126
(6126, 2)
(1, 6126, 2)


In [13]:
print(type(mesh.get_node_indice()))
print(len(mesh.get_node()))
print(mesh.get_node_indice())
node_indice_list=np.ndarray.tolist(mesh.get_node_indice())


<class 'numpy.ndarray'>
6126
[1.000e+00 2.000e+00 3.000e+00 ... 6.963e+03 6.964e+03 6.965e+03]


In [17]:

my_vec_solution = SolutionMat(
    label="my_vector",
    type_cell="triangle",
    field=temp,
    indice=node_indice_list, # optional if indice are sorted and starts from 0, but field size must match with the number of corresponding point/cell.
    axis_name=["time", "indice", "component"],
    axis_size = [1, 6500, 2],
)
MS.solution.append(my_vec_solution)
MS.plot_glyph(label="my_vector", is_point_arrow=True, factor=1/50)

6126

# Demo with FEMM results
The aim of this section is to show how MeshSolution object are used in Pyleecan to post-process FE results. 

In [47]:
# Run the FEMM simulation such as in tuto_Simulation_FEMM
import json
from multiprocessing import cpu_count
from os.path import join

import matplotlib.pyplot as plt
import numpy as np
import pytest
from numpy import array, ones, pi, zeros
from pyleecan.Classes.ImportGenVectLin import ImportGenVectLin
from pyleecan.Classes.ImportMatrixVal import ImportMatrixVal
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.MagFEMM import MagFEMM
from pyleecan.Classes.Output import Output
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.definitions import DATA_DIR
from pyleecan.Functions.load import load
from Tests import save_load_path, save_plot_path

SPMSM_003 = load(join(DATA_DIR, "Machine", "SPMSM_003.json"))
simu = Simu1(name="test_SIPMSM_003", machine=SPMSM_003)

# Definition of the enforced output of the electrical module
N0 = 3000
Is = ImportMatrixVal(
    value=array(
        [
            [6.97244193e-06, 2.25353053e02, -2.25353060e02],
            [-2.60215295e02, 1.30107654e02, 1.30107642e02],
            [-6.97244208e-06, -2.25353053e02, 2.25353060e02],
            [2.60215295e02, -1.30107654e02, -1.30107642e02],
        ]
    )
)
time = ImportGenVectLin(start=0, stop=0.015, num=4, endpoint=True)
Na_tot = 1024

simu.input = InputCurrent(
    Is=Is,
    Ir=None,  # No winding on the rotor
    OP=OPdq(N0=N0),
    time=time,
    Na_tot=Na_tot,
    angle_rotor_initial=0.5216 + pi,
)


To enable the FE results saving: is_get_mesh 

In [48]:
# Definition of the magnetic simulation (no symmetry)
simu.mag = MagFEMM(
    type_BH_stator=1,
    type_BH_rotor=1,
    is_periodicity_a=True,
    is_get_meshsolution=True,
    nb_worker=cpu_count(),
)
out_femm = simu.run()

[03:41:45] Starting running simulation test_SIPMSM_003 (machine=SPMSM_003)
[03:41:45] Enforcing outelec.phase_dir=1 to comply with input current
[03:41:45] Starting Magnetic module
[03:41:46] Solving time step 1 / 4 in FEMM
[03:41:47] Solving time step 2 / 4 in FEMM
[03:41:47] Solving time step 3 / 4 in FEMM
[03:41:48] Solving time step 4 / 4 in FEMM
[03:41:51] End of simulation test_SIPMSM_003


C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\lib\multiprocessing\pool.py:265: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.ThreadPool state=RUN pool_size=4>
  _warn(f"unclosed running multiprocessing pool {self!r}",
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x23ca21070a0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x23ca2107ca0>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x23ca2107d00>
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x23ca21073a0>


Now, the magnetic FEA results can be plotted. Moreover, the solution can be extracted on a specific area. By default, the field is plotted on the first dimension of every additional axis (e.g. time).

In [45]:
out_femm.mag.meshsolution.plot_contour()

C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\lib\site-packages\vtkmodules\util\numpy_support.py:74: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,


In [14]:
out_femm.mag.meshsolution.plot_contour(label="H", group_names="stator core")

In [47]:
out_femm.mag.meshsolution.plot_glyph(label="B", group_names="stator core")

C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\lib\site-packages\pyvista\core\dataset.py:459: PyvistaDeprecationWarning: Use of `DataSet.vectors` to add vector data is deprecated. Use `DataSet['vector_name'] = data`. Use `DataSet.active_vectors_name = 'vector_name' to make active.
  warnings.warn(


In [10]:
out_femm.mag.meshsolution.plot_contour(label="B", group_names="stator core",is_animated=True,save_path="D:Stator_Core.gif",clim=[0,2] )

알고리즘은 두 그룹 간의 인터페이스를 추출할 수도 있습니다.
The algorithm is even capable of extracting the interface between two groups:

In [18]:
out_femm.mag.meshsolution.plot_mesh(group_names=["stator core", "/", "airgap", "stator winding"])

None


In [42]:
out_femm.mag.meshsolution.plot_mesh(group_names=["rotor core", "stator winding", "airgap"])

NameError: name 'out_femm' is not defined

추가 축을 통한 슬라이스는 [SciDataTool](https://github.com/Eomys/SciDataTool) 유사 호출을 사용하여 얻을 수 있습니다. 예를 들어, 두 번째 시간 단계에서 자기장을 원하는 경우:

Slices through addtional axes can be obtained using [SciDataTool](https://github.com/Eomys/SciDataTool) alike call. For example, if I want the magnetic field at the second time step:

In [29]:
out_femm.mag.meshsolution.plot_contour(
    "time[2]",
    label="H",
    save_path=join(save_plot_path, simu.name + "_H_time2.png"),
    is_show_fig=False,
)

# 데이터 추출 및 후처리

개체 유형에 관계 없이 결과를 로드하기 위해 MeshSolution 클래스에 대해 여러 메서드가 개발되었습니다. MeshSolution의 주요 메소드는 get_field입니다. 첫 번째 인수는 [SciDataTool](https://github.com/Eomys/SciDataTool) 호출과 유사하게 작성할 수 있습니다. 예를 들어 FEMM 계산 후 첫 번째 단계에서 자속 밀도를 얻으려면 다음을 수행하십시오.
# Extract and post-process data

Several methods have been developed for the MeshSolution class in order to load the results regardless of the type of objects. The main method of MeshSolution is get_field. The first arguments can be written similarly to [SciDataTool](https://github.com/Eomys/SciDataTool) call. For example if I want to get the magnetic flux density at the first time step after FEMM calculations:

In [49]:
B = out_femm.mag.meshsolution.get_field("time[0]", "indice", "component", label='B')
B.shape

(26122, 2)

또 다른 유용한 기능은 그룹입니다. 그룹을 통해 기계의 하위 부분에 해당하는 하위 메시를 정의할 수 있습니다. 그룹 정의에서 새 MeshSolution 객체를 생성할 수 있습니다.

Another useful feature is the groups. The groups allows to define submeshes corresponding to subpart of the machine. A new MeshSolution object can be created from the group definition:

In [50]:
group_stator = out_femm.mag.meshsolution.get_group("stator core")


그런 다음 선택한 셀(이 경우 삼각형 요소)에서 해가 필터링됩니다.
Then, the magnetic solution is filtered on the selected cells (triangular elements in this case). 

In [53]:
B_s = group_stator.get_field("indice", "time", "component", label='B')
H_s = group_stator.get_field("indice", "time", "component", label='H')
H_s.shape
B_s.shape
w_mag = np.multiply(B_s,H_s)/2
w_mag.shape


(4, 1952, 2)

필드의 크기는 셀의 고정자 수로 분명히 축소되었습니다. 그런 다음 이 그룹의 솔루션에 대해 작업을 수행하고 플롯할 수 있습니다. 여러 유형의 Solution 객체가 동일한 MeshSolution 객체에 공존할 수 있다는 점은 주목할 가치가 있습니다.

The dimension of the field has been obviously reduced to the stator number of cells. Then, operations can be performed on the solution of this group, and plotted. It is worth noting that several type of Solution objects can co-exist in the same MeshSolution object.

In [58]:
B_s

array([[[ 1.20336945,  0.25761163],
        [ 1.22365736,  0.26430045],
        [ 1.2309031 ,  0.26222727],
        ...,
        [ 2.44811149,  2.15484842],
        [ 4.19862029,  0.80022394],
        [ 3.37502383,  2.01347532]],

       [[-1.04103604,  0.22625904],
        [-1.05867636,  0.22044312],
        [-1.06511704,  0.22228595],
        ...,
        [-1.78877178, -1.36445675],
        [-3.01473992, -0.41574603],
        [-2.40702053, -1.31098591]],

       [[-1.20336945, -0.25761163],
        [-1.22365736, -0.26430045],
        [-1.2309031 , -0.26222727],
        ...,
        [-2.44811149, -2.15484842],
        [-4.19862029, -0.80022394],
        [-3.37502383, -2.01347532]],

       [[ 1.04103604, -0.22625904],
        [ 1.05867636, -0.22044312],
        [ 1.06511704, -0.22228595],
        ...,
        [ 1.78877178,  1.36445675],
        [ 3.01473992,  0.41574603],
        [ 2.40702053,  1.31098591]]])

In [68]:
from pyleecan.Classes.SolutionMat import SolutionMat
w_mag = np.multiply(B_s,H_s)/2
#w_axis=dict()
#w_axis["time"]=16
#w_axis["indice"]=3801
#w_axis["component"]=2
my_vec_solution = SolutionMat(
    label="w_mag",
    type_cell="triangle",
    field=w_mag,
    axis_name=["time","indice", "component"],
    axis_size = [4, 1360,2],
   # axis_size = [4, 1952, 2],
)
group_stator.solution.append(my_vec_solution)
group_stator.plot_contour(label="w_mag")



#MSol.plot_mesh()

C:\Users\KDH2018-PC\anaconda3\envs\py38_openturn\lib\site-packages\vtkmodules\util\numpy_support.py:74: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  _vtk_np = {vtkConstants.VTK_BIT:numpy.bool,


Operations can also be performed on the mesh.

In [69]:
nodes_s = group_stator.get_mesh().get_node()
nodes_s.shape

(1360, 2)

Then, rotate the mesh

In [70]:
th = np.pi/2
R = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
nodes_s = np.dot(nodes_s, R)
group_stator.mesh[0].node.coordinate = nodes_s
group_stator.plot_mesh()

None


Previous plots still work !

In [71]:
group_stator.plot_contour(label="w_mag")

Thanks for following this tutorial ! :-)